In [ ]:
import numpy as np
import pandas as pd
import gc

In [ ]:
def changeDType(df,flag=False):
    
    if(flag):
        numericDtype = ['int32','int64','float64','float32']
    
    for i in df.columns:
        if (df[i].dtype == 'int64' or df[i].dtype == 'int32'):
            df[i] = pd.to_numeric(df[i],downcast='integer')
        
        if (df[i].dtype == 'float64' or df[i].dtype == 'float32'):
            df[i] = pd.to_numeric(df[i],downcast='float')

In [ ]:
dfTrain = pd.read_csv('../input/proprocessedtrain/preprocessesTrain.csv')
dfTest = pd.read_csv('../input/testdata/preprocessedTest.csv')

In [ ]:
changeDType(dfTrain)
changeDType(dfTest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [ ]:
def randomForest(df,isTest = False,model=None):
    
    train,test = train_test_split(df,test_size=0.2,stratify=df['isFraud'],random_state=9)
    
    rf = RandomForestClassifier(n_estimators=100,criterion="gini",bootstrap=False,n_jobs=-1,verbose=1)
    
    x = train.drop(['isFraud'],axis=1)
    y = train['isFraud']
   
    x = x-x.min()/x.max()-x.min()
    
    rf.fit(x,y)
    
    xt = test.drop(['isFraud'],axis=1)
    yt = test['isFraud']
    
    print("Accuracy : ",rf.score(xt,yt))
    
    prob = rf.predict_proba(xt)[:,1]
    
    auc = metrics.roc_auc_score(yt,prob)
    print("AUC : ",auc)
    return [rf,auc]

In [ ]:
def predict(df,model):
    
    x = df
    x = x - x.min()/x.max()-x.min()
    
    y_pred = model.predict(x)
    
    result = pd.DataFrame(y_pred)
    
    result.columns = ['isFraud']
    
    result.to_csv('./result_rf_wo_tunning.csv')
    print("Result saved")
    return
    
    

In [ ]:
model = randomForest(dfTrain)

In [ ]:
predict(dfTest,model[0])